In [1]:
import pandas as pd
import numpy as np
# import pm4py
import copy
import random
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
#1.加载模型

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error,  mean_squared_error,r2_score,median_absolute_error,explained_variance_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost.sklearn import XGBRegressor
from lightgbm.sklearn import LGBMRegressor
#模型实例化
# model = LinearRegression(normalize=True)#其他模型类似
# #向模型中填充数据
# model = model.fit(train_X, train_Y)
#模型预测

In [2]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.decomposition import PCA

In [3]:
data1=pd.read_csv("shuju_model_road.csv",index_col=0)

In [4]:
data1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,label,fitness,fitness_token
0,Send Fine,Appeal to Judge,Insert Fine Notification,Receive Result Appeal from Prefecture,Notify Result Appeal to Offender,Add penalty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.875000,0.950000
1,Create Fine,Send Appeal to Prefecture,Insert Fine Notification,Send Appeal to Prefecture,Receive Result Appeal from Prefecture,Appeal to Judge,Insert Fine Notification,Receive Result Appeal from Prefecture,Send Fine,NaN,NaN,NaN,NaN,NaN,NaN,0,0.818182,0.858238
2,Create Fine,Appeal to Judge,Appeal to Judge,Send Fine,Insert Fine Notification,Payment,Payment,Receive Result Appeal from Prefecture,Add penalty,Insert Fine Notification,Payment,Send Fine,Receive Result Appeal from Prefecture,Send for Credit Collection,Send for Credit Collection,0,0.882353,0.982759
3,Payment,Appeal to Judge,Send Appeal to Prefecture,Add penalty,Receive Result Appeal from Prefecture,Send Fine,Payment,Send Appeal to Prefecture,Add penalty,Notify Result Appeal to Offender,NaN,NaN,NaN,NaN,NaN,0,0.916667,0.973684
4,Create Fine,Appeal to Judge,Send Appeal to Prefecture,Add penalty,Send for Credit Collection,Insert Fine Notification,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.875000,0.950000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33239,Create Fine,Appeal to Judge,Insert Fine Notification,Send for Credit Collection,Send Appeal to Prefecture,Payment,Payment,Send Appeal to Prefecture,Send for Credit Collection,Appeal to Judge,Insert Fine Notification,Send for Credit Collection,NaN,NaN,NaN,1,1.000000,1.000000
33240,Create Fine,Insert Date Appeal to Prefecture,Insert Fine Notification,Send for Credit Collection,Payment,Appeal to Judge,Send Appeal to Prefecture,Send Fine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.000000,1.000000
33241,Create Fine,Insert Date Appeal to Prefecture,Send Appeal to Prefecture,Receive Result Appeal from Prefecture,Appeal to Judge,Insert Date Appeal to Prefecture,Send Appeal to Prefecture,Receive Result Appeal from Prefecture,Send for Credit Collection,Insert Date Appeal to Prefecture,NaN,NaN,NaN,NaN,NaN,1,1.000000,1.000000
33242,Create Fine,Insert Date Appeal to Prefecture,Send Fine,Send Fine,Appeal to Judge,Insert Fine Notification,Appeal to Judge,Add penalty,Notify Result Appeal to Offender,Send Appeal to Prefecture,NaN,NaN,NaN,NaN,NaN,1,1.000000,1.000000


In [5]:
def metric(data):
    fitness=data["fitness"]
    token_fitness=data["fitness_token"]
    print("mse",mean_squared_error(fitness,token_fitness))
    print("mae",mean_absolute_error(fitness,token_fitness))
    print("r2",r2_score(fitness,token_fitness))

In [6]:
metric(data1)

mse 0.0034182600489682763
mae 0.02735635059724609
r2 0.5679816787243204


In [7]:
new_labels=data1["label"]
data1.drop(columns=['label'],inplace=True)
new_fitness=data1["fitness"]
data1.drop(columns=["fitness"],inplace=True)
data1.drop(columns=["fitness_token"],inplace=True)

In [8]:
dataset=data1

In [9]:
dataset=dataset.replace(['Insert Fine Notification', 'Insert Date Appeal to Prefecture', 'Receive Result Appeal from Prefecture', 'Add penalty', 'Send Fine', 'Payment', 'Send for Credit Collection', 'Notify Result Appeal to Offender', 'Send Appeal to Prefecture', 'Appeal to Judge', 'Create Fine'],[0,1,2,3,4,5,6,7,8,9,10])

In [10]:
dataset=dataset.fillna(11)

In [11]:
# 3.1 数据类型转换
dataset_tensor = torch.Tensor(dataset.values)

# 对datafram 进行独热编码

In [12]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
num_trace=data1.shape[0]
len_trace=dataset.shape[1]
num_unique=12

dd=torch.zeros(num_trace, len_trace,num_unique, dtype=torch.long)
#dd(num_trace,len_trace,num_unique)
def one_hot(x, n_class, dtype=torch.float32): 
    # X shape: (batch), output shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res
def to_onehot(X, n_class):  
    # X shape: (batch, seq_len), output: seq_len elements of (batch, n_class)
    for i in range(X.shape[0]):
        dd[i,:,:]=one_hot(X[i,:], n_class) 
    return dd



In [13]:
ceshi=to_onehot(dataset_tensor,num_unique)

# 增加一个batch_size维度

In [14]:
ceshi=ceshi.view(ceshi.shape[0],-1)

In [15]:
ceshi=ceshi.numpy()

In [16]:
ceshi=pd.DataFrame(ceshi)

In [17]:
new_fitness

0        0.875000
1        0.818182
2        0.882353
3        0.916667
4        0.875000
           ...   
33239    1.000000
33240    1.000000
33241    1.000000
33242    1.000000
33243    1.000000
Name: fitness, Length: 33244, dtype: float64

In [18]:
ceshi

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33239,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
33240,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
33241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
33242,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
dd=ceshi

# 使用pac降维

In [20]:

pca = PCA(n_components=56)
pca.fit(dd)
ceshi=pca.transform(dd)

In [21]:
from sklearn.model_selection import KFold

In [36]:
kf = KFold(n_splits=5,shuffle=True,random_state=1)

In [37]:
new_fitness

0        0.875000
1        0.818182
2        0.882353
3        0.916667
4        0.875000
           ...   
33239    1.000000
33240    1.000000
33241    1.000000
33242    1.000000
33243    1.000000
Name: fitness, Length: 33244, dtype: float64

In [38]:
list_a=[]
for train_index, test_index in kf.split(ceshi):
#     print('train_index', train_index, 'test_index', test_index)
    x_train, y_train = ceshi[train_index], new_fitness[train_index]
    x_test, y_test = ceshi[test_index], new_fitness[test_index]
    models = [DecisionTreeRegressor(random_state=1)]
    asses_method=['mean_absolute_error','mean_squared_error','r2_score']
    #将各模型结果保存在字典中
    result =[[]*i for i in range(0,len(models))]
    i=-1
    for model in models:
        i=i+1
#         print(model)
        for ass_method in asses_method:
                model.fit(x_train, y_train)
                score = eval(ass_method)(y_test, model.predict(x_test))
#                 print(score)
                result[i].append(score)
#     print(result)
    list_a.append(result)
#     ass_data = pd.DataFrame(result) 
#     ass_data.rename(columns={0:"mean_absolute_error",1:"mean_squared_error"},inplace=True)
#     ass_data.index=["DecisionTreeRegressor"]
#     ass_data
cca=np.zeros(3)
for i in range(0,len(list_a)):
    cca= np.array(list_a[i][0])+cca
print(cca/5)

[0.03933462 0.00540144 0.31711557]


In [39]:
# from sklearn.cluster import AgglomerativeClustering
# from collections import Counter
# km = AgglomerativeClustering(n_clusters=3).fit(ceshi)

In [40]:
from sklearn.cluster import KMeans
from collections import Counter
km = KMeans(n_clusters=3).fit(ceshi)

In [41]:
# from sklearn.cluster import Birch
# from collections import Counter
# km = Birch(n_clusters=3).fit(ceshi)

In [42]:
count = Counter(km.labels_)

In [43]:
list_temp=[]
for i in range(3):
    list_=ceshi[np.where(km.labels_==i)[0]]
    label_=new_fitness[np.where(km.labels_==i)[0]]
    kf = KFold(n_splits=5,shuffle=True,random_state=1)
    list_a=[]
    label_.reset_index(drop=True,inplace=True)
    for train_index, test_index in kf.split(list_):
        x_train, y_train = list_[train_index], label_[train_index]
        x_test, y_test = list_[test_index], label_[test_index]
        models = [DecisionTreeRegressor(random_state=1)]
        asses_method=['mean_absolute_error','mean_squared_error','r2_score']
        #将各模型结果保存在字典中
        result =[[]*i for i in range(0,len(models))]
        i=-1
        for model in models:
            i=i+1
    #         print(model)
            for ass_method in asses_method:
                model.fit(x_train, y_train)
                score = eval(ass_method)(y_test, model.predict(x_test))
                print(ass_method)
                print(score)
                result[i].append(score)
        list_a.append(result)
#         print(result)
    ccb=np.zeros(3)
    for i in range(0,len(list_a)):
        ccb= np.array(list_a[i][0])+ccb
#     print(ccb/5)
    list_temp.append((ccb/5).tolist())
    #     break

mean_absolute_error
0.04039611213599952
mean_squared_error
0.004987578145635195
r2_score
-0.34452705520296845
mean_absolute_error
0.04002802695209598
mean_squared_error
0.0047745990110935905
r2_score
-0.5242855215943261
mean_absolute_error
0.042162929649217
mean_squared_error
0.005532876868435712
r2_score
-0.5396104673142448
mean_absolute_error
0.04153013077904859
mean_squared_error
0.005105770102547991
r2_score
-0.4588004334330045
mean_absolute_error
0.04196304355333944
mean_squared_error
0.005113780210887361
r2_score
-0.428550275350303
mean_absolute_error
0.037526633522727275
mean_squared_error
0.004768626876004361
r2_score
0.05688264278479693
mean_absolute_error
0.03621189915767346
mean_squared_error
0.004760583810301801
r2_score
0.007718160262727203
mean_absolute_error
0.03684623469674765
mean_squared_error
0.0046164491654565815
r2_score
-0.0037188944918848943
mean_absolute_error
0.0379254193128155
mean_squared_error
0.005133887617122505
r2_score
-0.06609035861617185
mean_absolute_

In [44]:
dict1=dict(count)

In [45]:
list_temp

[[0.04121604861394011, 0.00510292086771997, -0.4591547505789693],
 [0.03715610907638303, 0.004855941568308391, 0.0040777088765175055],
 [0.036885021374606294, 0.005862285123732252, 0.5276518241044794]]

In [46]:
cc0=pd.DataFrame(list_temp[0])
cc1=pd.DataFrame(list_temp[1])
cc2=pd.DataFrame(list_temp[2])
# cc3=pd.DataFrame(list_temp[3])

In [47]:
cc0*dict1[0]/sum(dict1.values())+cc1*dict1[1]/sum(dict1.values())+cc2*dict1[2]/sum(dict1.values())

,0
0,0.038261
1,0.005326
2,0.071948


In [48]:
print(cca/5)

[0.03933462 0.00540144 0.31711557]


# ----------------------------------------------------------

# -------------------------------------------------------

In [33]:
x_train, x_test, y_train, y_test = train_test_split(ceshi, new_fitness, random_state=0, train_size=0.8)

In [49]:
models = [DecisionTreeRegressor(),
          RandomForestRegressor(),
          GradientBoostingRegressor(),
          MLPRegressor(solver='lbfgs', max_iter=1000),
          XGBRegressor(n_estimators = 1000, objective='reg:squarederror'), 
          LGBMRegressor(n_estimators = 1000)]
asses_method=['mean_absolute_error','mean_squared_error','r2_score','median_absolute_error','explained_variance_score']
#将各模型结果保存在字典中
result =[[]*i for i in range(0,len(models))]
i=-1
for model in models:
    i=i+1
    print(model)
    for ass_method in asses_method:

            model.fit(x_train, y_train)
            score = eval(ass_method)(y_test, model.predict(x_test))
            print(score)
            result[i].append(score)

ass_data = pd.DataFrame(result) 
ass_data.rename(columns={0:"mean_absolute_error",1:"mean_squared_error",2:"r2_score",3:"median_absolute_error",4:"explained_variance_score"},inplace=True)
ass_data.index=["DecisionTreeRegressor","RandomForestRegressor","GradientBoostingRegressor","MLPRegressor","XGBRegressor","LGBMRegressor"]

DecisionTreeRegressor()
0.03804944471738513
0.005994963245916265
0.5098857170354303
0.0
0.516084948599586
RandomForestRegressor()
0.035269688119802765
0.0028298130289368577
0.7655591300790014
0.020000000000001017
0.7703517001088553
GradientBoostingRegressor()
0.04606403461837193
0.003726417655549667
0.6976429876441579
0.03375645686210149
0.6974477632123968
MLPRegressor(max_iter=1000, solver='lbfgs')
0.0385207192574767
0.0027576347238503357
0.7669568310738455
0.026684428902002733
0.7714953341106897
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None

In [50]:
ass_data

,mean_absolute_error,mean_squared_error,r2_score,median_absolute_error,explained_variance_score
DecisionTreeRegressor,0.038049,0.005995,0.509886,0.000000,0.516085
RandomForestRegressor,0.035270,0.002830,0.765559,0.020000,0.770352
GradientBoostingRegressor,0.046064,0.003726,0.697643,0.033756,0.697448
MLPRegressor,0.038521,0.002758,0.766957,0.026684,0.771495
XGBRegressor,0.032156,0.002442,0.801404,0.017328,0.801413
LGBMRegressor,0.028873,0.002098,0.829406,0.014098,0.829411


# claster

# KMeans划分聚类

In [51]:
from collections import Counter

In [64]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3).fit(ceshi)

In [65]:
count = Counter(km.labels_)

In [66]:
count

Counter({0: 13087, 2: 10236, 1: 9921})

# Birch聚类

In [22]:
# from sklearn.cluster import Birch
# gas= Birch(n_clusters=3).fit(ceshi)
# count = Counter(gas.labels_)
# count

# MiniBatchKMeans

In [23]:
# from sklearn.cluster import  MiniBatchKMeans
# clustering = MiniBatchKMeans(n_clusters=3).fit(ceshi)

In [24]:
# count = Counter(clustering.labels_)
# count

In [69]:
len(new_fitness)

33244

In [70]:
list_temp=[]
for i in range(3):
    list_=ceshi[np.where(km.labels_==i)[0]]
    label_=new_fitness[np.where(km.labels_==i)[0]]
    label_.reset_index(drop=True,inplace=True)
    x_train, x_test, y_train, y_test = train_test_split(list_,label_, random_state=0, train_size=0.8)
    models = [DecisionTreeRegressor(),
          RandomForestRegressor(),
          GradientBoostingRegressor(),
          MLPRegressor(solver='lbfgs', max_iter=1000),
          XGBRegressor(n_estimators = 1000, objective='reg:squarederror'), 
          LGBMRegressor(n_estimators = 1000)]
    asses_method=['mean_absolute_error','mean_squared_error','r2_score']
    #将各模型结果保存在字典中
    result =[[]*i for i in range(0,len(models))]
    i=-1
    for model in models:
        i=i+1
#         print(model)
        for ass_method in asses_method:
                model.fit(x_train, y_train)
                score = eval(ass_method)(y_test, model.predict(x_test))
#                 print(score)
                result[i].append(score)
#     print(result)
#     ass_data = pd.DataFrame(result)
#     ass_data.rename(columns={0:"mean_absolute_error",1:"mean_squared_error",2:"r2_score"},inplace=True)
#     ass_data.index=["DecisionTreeRegressor","RandomForestRegressor","GradientBoostingRegressor","MLPRegressor","XGBRegressor","LGBMRegressor"]
#     data_test=ass_data.copy()
#     data_test=data_test+ass_data
#     print(ass_data)
    list_temp.append(result)
#     break

In [71]:
dict1=dict(count)

In [72]:
cc0=pd.DataFrame(list_temp[0])
cc1=pd.DataFrame(list_temp[1])
cc2=pd.DataFrame(list_temp[2])
# cc3=pd.DataFrame(list_temp[3])

In [63]:
cc0*dict1[0]/sum(dict1.values())+cc1*dict1[1]/sum(dict1.values())+cc2*dict1[2]/sum(dict1.values())

,0,1,2
0,0.037707,0.005375,0.081269
1,0.034829,0.002485,0.567244
2,0.039239,0.002872,0.528607
3,0.033146,0.002013,0.681713
4,0.031215,0.002135,0.633237
5,0.027908,0.001777,0.698457


In [58]:
cc0*dict1[0]/sum(dict1.values())+cc1*dict1[1]/sum(dict1.values())+cc2*dict1[2]/sum(dict1.values())

,0,1,2
0,0.038038,0.005394,0.077847
1,0.034955,0.002493,0.562047
2,0.039224,0.002873,0.529466
3,0.032605,0.002039,0.682606
4,0.031215,0.002135,0.633237
5,0.027908,0.001777,0.698457


In [59]:
ass_data

,mean_absolute_error,mean_squared_error,r2_score,median_absolute_error,explained_variance_score
DecisionTreeRegressor,0.038049,0.005995,0.509886,0.000000,0.516085
RandomForestRegressor,0.035270,0.002830,0.765559,0.020000,0.770352
GradientBoostingRegressor,0.046064,0.003726,0.697643,0.033756,0.697448
MLPRegressor,0.038521,0.002758,0.766957,0.026684,0.771495
XGBRegressor,0.032156,0.002442,0.801404,0.017328,0.801413
LGBMRegressor,0.028873,0.002098,0.829406,0.014098,0.829411
